In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("../../data/AmesHousing.csv")
# df = df.select_dtypes(include="number")
# df.dropna(inplace=True)
# df["random_feature"] = np.random.random(df.shape[0])
# df["Overall Qual Noise"] = df["Overall Qual"] + (df["random_feature"] * 0.0001)
# Split the data
test_size = 0.3
label = "SalePrice"

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(label, axis=1), df[label], test_size=test_size, random_state=42
)

In [5]:
import autogluon

In [6]:
train_data = TabularDataset(X_train)
train_data[label] = y_train
test_data = TabularDataset(X_test)
test_data[label] = y_test

In [108]:
from autogluon.common import space
import numpy as np

nn_options = {  # specifies non-default hyperparameter values for neural network models
    "y_range": (0, np.inf),
    "num_layers": space.Categorical(3, 4), # 3, 4),
    "learning_rate": space.Categorical(1e-2), #1e-2, 1e-3, 3e-3),
    "hidden_size": space.Categorical(128, 256, 512),#, 64, 256, 512), #64, 128, 256, 
    "max_batch_size" : space.Categorical(128, 256, 512), #512
    "proc.embed_min_categories" : 4,
    "dropout_prob" : space.Categorical(0.01),
    #'activation': space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'use_batchnorm' : space.Categorical(False), #, True),  # activation function used in NN (categorical hyperparameter, default = first entry)
    #'dropout_prob': space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
    
    # ## Hyperparameters for neural net architecture:
    "activation": "relu",  # Activation function
    # Options: ['relu', 'softrelu', 'tanh', 'softsign'], options for pytorch: ['relu', 'elu', 'tanh']
    "embedding_size_factor": 1.0,  # scaling factor to adjust size of embedding layers (float > 0)
    # Options: range[0.01 - 100] on log-scale
    "embed_exponent": 0.56,  # exponent used to determine size of embedding layers based on # categories.
    "max_embedding_dim": 100,  # maximum size of embedding layer for a single categorical feature (int > 0).
    ## Regression-specific hyperparameters:
    #"y_range": None,  # Tuple specifying whether Y is constrained to (min_y, max_y). Can be = (-np.inf, np.inf).
    # If None, inferred based on training labels. Note: MUST be None for classification tasks!
    "y_range_extend": 0.05,  # Only used to extend size of inferred y_range when y_range = None.
    ## Hyperparameters for neural net training:
    #"dropout_prob": 0.1,  # dropout probability, = 0 turns off Dropout.
    # # Options: range(0.0, 0.5)
    "optimizer": "adam",  # Which optimizer to use for training
    # Options include: ['adam','sgd']
    #"learning_rate": 3e-4,  # learning rate used for NN training (float > 0)
    "weight_decay": 1e-6,  # weight decay regularizer (float > 0)
    ## Hyperparameters for data processing:
    "proc.embed_min_categories": 4,  # apply embedding layer to categorical features with at least this many levels. Features with fewer levels are one-hot encoded. Choose big value to avoid use of Embedding layers
    # Options: [3,4,10, 100, 1000]
    "proc.impute_strategy": "median",  # strategy argument of sklearn.SimpleImputer() used to impute missing numeric values
    # Options: ['median', 'mean', 'most_frequent']
    "proc.max_category_levels": 100,  # maximum number of allowed levels per categorical feature
    # Options: [10, 100, 200, 300, 400, 500, 1000, 10000]
    "proc.skew_threshold": 0.99,  # numerical features whose absolute skewness is greater than this receive special power-transform preprocessing. Choose big value to avoid using power-transforms
    # Options: [0.2, 0.3, 0.5, 0.8, 0.9, 0.99, 0.999, 1.0, 10.0, 100.0]
    "use_ngram_features": False,  # If False, will drop automatically generated ngram features from language features. This results in worse model quality but far faster inference and training times.
    # Options: [True, False]
    
    # #"num_layers": 4,  # number of layers
    # # Options: [2, 3, 4, 5]
    # #"hidden_size": 128,  # number of hidden units in each layer
    # # Options: [128, 256, 512]
    # #"max_batch_size": 512,  # maximum batch-size, actual batch size may be slightly smaller.
    # "use_batchnorm": False,  # whether or not to utilize batch normalization
    # # Options: [True, False]
    # "loss_function": "auto",  # Pytorch loss function minimized during training
    # # Example options for regression: nn.MSELoss(), nn.L1Loss()
    
}

In [134]:
time_limit = 2 * 60  # train various models for ~2 min
num_trials = 2*3*3

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    "num_trials": num_trials,
    "scheduler": "local",
    "searcher": "random",
}

In [135]:
%%time
predictor = TabularPredictor(label=label, eval_metric="mean_absolute_percentage_error")
predictor.fit(train_data=train_data.head(1000), verbosity=2, 
              hyperparameters = {"NN_TORCH" : nn_options},
              hyperparameter_tune_kwargs = hyperparameter_tune_kwargs,
              included_model_types=["NN_TORCH"]
              )

No path specified. Models will be saved in: "AutogluonModels/ag-20240427_192908"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240427_192908"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT

	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 3 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 11 | ['Lot Frontage', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', ...]
		('int', [])    : 27 | ['Order', 'PID', 'MS SubClass', 'Lot Area', 'Overall Qual', ...]
		('object', []) : 43 | ['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', [])  : 40 | ['MS Zoning', 'Alley', 'Lot Shape', 'Land Contour', 'Lot Config', ...]
		('fl

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
predictions = predictor.predict(test_data)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
mean_absolute_percentage_error(y_test, predictions)

0.08516899829912311

In [ ]:
results = predictor.fit_summary(verbosity=10, show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val                     eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.092838  mean_absolute_percentage_error       0.046738  12.100504                0.000280           0.015348            2       True         25
1     NeuralNetTorch/T5  -0.097181  mean_absolute_percentage_error       0.014749   6.563594                0.014749           6.563594            1       True          5
2     NeuralNetTorch/T4  -0.097181  mean_absolute_percentage_error       0.015342   6.620213                0.015342           6.620213            1       True          4
3     NeuralNetTorch/T3  -0.097726  mean_absolute_percentage_error       0.014119   2.632468                0.014119           2.632468            1       True          3
4    NeuralNetTorch/T15  -0.097726  mean_absolute_percentage_error       0.015022  

/Users/giladrubin/mambaforge/envs/ml-env/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning:

AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"



In [ ]:
df_hyperparams = pd.DataFrame.from_dict(results["model_hyperparams"], orient='index')
consistent_hyperparams = df_hyperparams.nunique()[df_hyperparams.nunique() == 1].index
df_hyperparams = df_hyperparams.drop(columns=consistent_hyperparams)
df_hyperparams = df_hyperparams.reset_index().rename({"index" : "model"}, axis=1).copy()

In [ ]:
leaderboard = results["leaderboard"]
df = leaderboard.merge(df_hyperparams, on="model")

In [ ]:
import plotly.express as px

In [ ]:
px.scatter(df, x="fit_time", y="score_val", color='model', text="model")

In [ ]:
# We identify all unique hyperparameters across models
variable_hyperparams = set(df_hyperparams.columns) - {'model', 'score_val', 'fit_time', "fit_order"}

# Iterate over each hyperparameter
for col in variable_hyperparams:
    if df[col].dtype == 'object' or df[col].dtype == 'bool':
        # Use a box plot for categorical data
        fig = px.box(df, x=col, y='score_val', color='model', title=f'Effect of {col} on Score Validation')
        fig.show()
        fig = px.box(df, x=col, y='fit_time', color='model', title=f'Effect of {col} on Fit Time')
        fig.show()
    else:
        # Use scatter plot for numerical data
        fig = px.scatter(df, x=col, y='score_val', color='model', title=f'Effect of {col} on Score Validation')
        fig.show()
        fig = px.scatter(df, x=col, y='fit_time', color='model', title=f'Effect of {col} on Fit Time')
        fig.show()
